In [40]:
from azureml.core import Workspace
import azureml.core

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 0.1.0.1005480


In [42]:
ws = Workspace.get_or_create(name='mywsprod2', location='eastus2euap', subscription_id='6a275e9b-a3e6-42c2-b158-760d27f5cb83', resource_group='marinchprodrg2')
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: mywsprod2
Azure region: eastus2euap
Subscription id: 6a275e9b-a3e6-42c2-b158-760d27f5cb83
Resource group: marinchprodrg2


In [43]:
# get the text data from the github repo and unzip it
from fit_and_store_pipeline import unzip_file_here
import urllib
import os

if not os.path.isfile('./attack_data.csv'):
    if not os.path.isfile('./text_data.zip'): 
        urllib.request.urlretrieve('https://activelearning.blob.core.windows.net/activelearningdemo/text_data.zip', 'text_data.zip')
    unzip_file_here('text_data.zip')

if not os.path.isfile('miniglove_6B_50d_w2v.txt'):
    unzip_file_here('miniglove_6B_50d_w2v.zip')
    
print('Data files here')

Data files here


In [44]:
from fit_and_store_pipeline import *
script_main()

[[861  14]
 [ 75  50]]
{"result": [1, 0, 0, 0, 1]}


In [46]:
%%writefile myenv.yml
name: myenv
channels:
  - defaults
dependencies:
  - pip:
    - numpy
    - scikit-learn

Overwriting myenv.yml


### Create `score.py` file
The `%%writefile` cell magic is used to write the scoring function to a local file. 

In [47]:
%%writefile score.py
import pickle
import json
import numpy
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier

def init():
    global model
    model = joblib.load("attack_model.pkl")

# note you can pass in multiple rows for scoring
def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = numpy.array(data)
        result = model.predict(data)
    except Exception as e:
        result = str(e)
    return json.dumps({"result": result.tolist()})

Writing score.py


In [48]:
# you may need to register the provider in your subscription 
# az provider register -n Microsoft.ContainerInstance

from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = ['MLADS'], 
                                               description = 'Toxicity scoring')

In [46]:
os.getcwd()

'C:\\Users\\marinch\\Source\\repos\\active-learning-workshop\\deployment'

In [49]:
%%time
# this will take 5-10 minutes to finish
# you can also use "az container list" command to find the ACI being deployed
service = ws.deploy_webservice(name = 'my-svc1',
                               deploy_config = aciconfig,
                               target = None,
                               models = ['attack_model.pkl'],
                               runtime = 'python',
                               conda_file = 'myenv.yml',
                               driver = 'score.py')
service.wait_for_deployment(show_output = True)

Registering model attack_model.pkl


Client-Request-ID=19303a06-6e7e-11e8-b270-dc53600a4933 Retry policy did not allow for a retry: Server-Timestamp=Tue, 12 Jun 2018 20:20:49 GMT, Server-Request-ID=04a41f5e-001e-0031-668a-02c4ba000000, HTTP status code=409, Exception=The specified container already exists.ErrorCode: ContainerAlreadyExists<?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.RequestId:04a41f5e-001e-0031-668a-02c4ba000000Time:2018-06-12T20:20:49.8977641Z</Message></Error>.
Client-Request-ID=1972c89c-6e7e-11e8-b503-dc53600a4933 Retry policy did not allow for a retry: Server-Timestamp=Tue, 12 Jun 2018 20:20:49 GMT, Server-Request-ID=9c218da2-101e-0060-598a-025936000000, HTTP status code=409, Exception=The specified container already exists.ErrorCode: ContainerAlreadyExists<?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.RequestId:9c218da2-101e-0060-598a-02593

Creating image
Image creation operation finished for image my-svc1:1, operation "Succeeded"
Creating service
Running...........................................................................
FailedACI service creation operation finished, operation "Failed"
Service creation failed, unexpected error response:
{'code': 'AciDeploymentFailed', 'message': 'Aci Deployment failed', 'details': [{'code': 'CrashLoopBackOff', 'message': 'CrashLoopBackOff: Back-off 20s restarting failed'}]}
Wall time: 10min 2s



## Test web service

In [53]:
print('web service is hosted in ACI:', service.scoring_uri)

web service is hosted in ACI: http://40.114.78.215:5001/score


Feed 100 rows of data to get predictions.

In [54]:
import pandas as pd

test_set_file = "test_set_01.csv"

test_data = pd.read_csv(test_set_file)

test_array = test_data.iloc[0:100, 1:51].as_matrix()

C:\Users\marinch\AppData\Local\Continuum\Miniconda3\envs\bugbashenv\lib\site-packages\ipykernel\__main__.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [55]:
import json

test_samples = json.dumps({"data": test_array.tolist()})
test_samples = bytes(test_samples, encoding = 'utf8')

# Call scoring service
service.run(input_data = test_samples)

'{"result": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}'

## Delete ACI to clean up

Deleting ACI is super fast!

In [37]:
%%time 
service.delete()

Wall time: 1.6 s


In [51]:
# Handy cleanup cell to delete all existing services 
from azureml.core.webservice import Webservice

services = Webservice.list(workspace=ws)
for i in range(0, len(services)):
    try:
        services[i].delete()
    except:
        pass

In [50]:
service.get_logs()

'2018-06-12 20:31:25,238 CRIT Supervisor running as root (no user in config file)\n2018-06-12 20:31:25,241 INFO supervisord started with pid 1\n2018-06-12 20:31:26,243 INFO spawned: \'rsyslog\' with pid 7\n2018-06-12 20:31:26,245 INFO spawned: \'program_exit\' with pid 8\n2018-06-12 20:31:26,250 INFO spawned: \'nginx\' with pid 9\n2018-06-12 20:31:26,342 INFO spawned: \'iot\' with pid 10\n2018-06-12 20:31:26,344 INFO spawned: \'gunicorn\' with pid 11\n2018-06-12 20:31:26,396 INFO success: iot entered RUNNING state, process has stayed up for > than 0 seconds (startsecs)\nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2018-06-12 20:31:26,486 INFO exited: iot (exit status 1; expected)\n2018-06-12 20:31:27,488 INFO success: rsyslog entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)\n2018-06-12 20:31:27,488 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)\n2018-06-12 20:31:31,492 